In [0]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [4]:
val_data = torchvision.datasets.ImageFolder('./drive/My Drive/cropped_image/validation',loader = plt.imread,transform=transform)
test_data = torchvision.datasets.ImageFolder('./drive/My Drive/cropped_image/test',loader = plt.imread,transform=transform)
train_data = torchvision.datasets.ImageFolder('./drive/My Drive/cropped_image/train',loader = plt.imread,transform=transform)

total_data = len(val_data) + len(test_data) + len(train_data)

print("Validation set percentage: "+str(len(val_data)/total_data))
print("Test set percentage: "+str(len(test_data)/total_data))
print("Training set percentage: "+str(len(train_data)/total_data))

Validation set percentage: 0.15677253826417764
Test set percentage: 0.05258191040726651
Training set percentage: 0.7906455513285559


In [0]:
import torchvision.models
#resnet_v1 = torchvision.models.resnet.ResNet(torchvision.models.resnet.Bottleneck,[3, 8, 36, 3],num_classes=32).cuda()

In [0]:
def get_train_accuracy(model,batch_size=20):
    
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,shuffle = True)

    correct = 0
    total = 0
    idx = 0
    
    for imgs, labels in iter(train_loader):
        if idx>20:
          break
        model = model.cuda()
        output = model(imgs.cuda()).cuda() # We don't need to run F.softmax
        pred = output.max(1, keepdim=True)[1].cuda() # get the index of the max log-probability
        correct += pred.eq(labels.cuda().view_as(pred)).sum().item()
        total += imgs.shape[0]
        idx += 1
        
    
    return correct / total

In [0]:
def get_val_accuracy(model,batch_size=20):
    
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size,shuffle = True)

    correct = 0
    total = 0
    idx = 0
    
    for imgs, labels in iter(val_loader):
        if idx>20:
          break
        model = model.cuda()
        output = model(imgs.cuda()).cuda() # We don't need to run F.softmax
        pred = output.max(1, keepdim=True)[1].cuda() # get the index of the max log-probability
        correct += pred.eq(labels.cuda().view_as(pred)).sum().item()
        total += imgs.shape[0]
        idx += 1

    return correct / total

In [0]:
def get_val_loss(model,batch_size=20):  

  val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size,shuffle = True)
  losses = []
  criterion = nn.CrossEntropyLoss().cuda()
  for imgs, labels in iter(val_loader):
      model = model.cuda()
      output = model(imgs.cuda()).cuda() # We don't need to run F.softmax
      loss = criterion(output, labels.cuda())
      losses.append(float(loss)/batch_size)
      break

  return sum(losses)/len(losses)

In [0]:
def train_net(model, batch_size=20, num_epochs=100, learning_rate = 0.0006,weight_decay=0.0002):
    checkpoint_num = 0
    model = model.cuda()
    model = model.train()
    start_time = time.time()
    
    torch.manual_seed(1000)
    train_loader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size=batch_size)
    criterion = nn.CrossEntropyLoss().cuda()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay = weight_decay)
    

    saved_x, saved_train_losses, saved_val_losses, train_acc, val_acc = [], [], [], [], []

    # training
    
    
    saved_idx = 0
    
    for epoch in range(num_epochs):
        idx = 0
        for imgs, labels in iter(train_loader):
            
            out = model(imgs.cuda()).cuda()             # forward pass
            loss = criterion(out, labels.cuda()) # compute the total loss


            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch
            print ('processing iterration '+str(idx)+'...')
            idx += 1
            if idx%200==0:
              saved_x.append(saved_idx)
              saved_idx += 1
              saved_val_losses.append(get_val_loss(model,batch_size))
              saved_train_losses.append(float(loss)/batch_size)
              print(("\tTrain loss: {} | Validation loss: {}\n").format(saved_train_losses[-1],saved_val_losses[-1]))

        
              train_acc.append(get_train_accuracy(model,batch_size=batch_size)) # compute training accuracy 
              val_acc.append(get_val_accuracy(model,batch_size=batch_size))  # compute validation accuracy


              print(("Epoch {}: Train acc: {} |"+"Validation acc: {}").format(epoch + 1,train_acc[-1],val_acc[-1]))
              
              model_path = "bs{0}_lr{1}_epoch{2}_checkpoint_{3}".format(batch_size,learning_rate,epoch,checkpoint_num)
              torch.save({
                  'checkpoint_num': checkpoint_num,
                  'epoch': epoch,
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  #'train_loss': train_losses,
                  #'val_loss':val_losses,
                  'saved_train_loss': saved_train_losses,
                  'saved_val_loss':saved_val_losses,
                  'saved_x':saved_x,
                  #'iters':iters,
                  'train_acc':train_acc,
                  'val_acc':val_acc
                  }, './drive/My Drive/resnet_model_saved/'+model_path+'.pth')


    # plotting
    plt.title("Training Curve")
    plt.plot(saved_x, saved_train_losses, label="Train")
    plt.plot(saved_x, saved_val_losses, label='Validation')
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()

    plt.title("Training Curve")
    plt.plot(saved_x, train_acc, label="Train")
    plt.plot(saved_x, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()
    
    end_time = time.time()
    duration = end_time - start_time

    print("Final Training Accuracy: {}".format(saved_train_acc[-1]))
    print("Final Validation Accuracy: {}".format(saved_val_acc[-1]))
    print ("Trained Duration: {} seconds".format(duration))

In [0]:
def train_net_continue(model, batch_size=20, num_epochs=100, learning_rate = 0.0006,weight_decay=0.0002):
    
    model_path = "bs{0}_lr{1}_epoch{2}_checkpoint_{3}".format(batch_size,learning_rate,0,0)
    checkpoint = torch.load('./drive/My Drive/resnet_model_saved/'+model_path+'.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    checkpoint_num = checkpoint['checkpoint_num']
    model = model.cuda()
    model = model.train()
    start_time = time.time()
    
    torch.manual_seed(1000)
    train_loader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size=batch_size)
    criterion = nn.CrossEntropyLoss().cuda()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay = weight_decay)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    

    saved_x = checkpoint['saved_x']
    saved_train_losses = checkpoint['saved_train_loss']
    saved_val_losses = checkpoint['saved_val_loss']
    train_acc = checkpoint['train_acc']
    val_acc = checkpoint['val_acc']
    
    # training
    
    
    saved_idx = saved_x[-1]+1
    
    for epoch in range(num_epochs):
        idx = 0
        for imgs, labels in iter(train_loader):
            
            out = model(imgs.cuda()).cuda()             # forward pass
            loss = criterion(out, labels.cuda()) # compute the total loss


            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch
            print ('processing iterration '+str(idx)+'...')
            idx += 1
            if idx%200==0:
              saved_x.append(saved_idx)
              saved_idx += 1
              saved_val_losses.append(get_val_loss(model,batch_size))
              saved_train_losses.append(float(loss)/batch_size)
              print(("\tTrain loss: {} | Validation loss: {}\n").format(saved_train_losses[-1],saved_val_losses[-1]))

        
              train_acc.append(get_train_accuracy(model,batch_size=batch_size)) # compute training accuracy 
              val_acc.append(get_val_accuracy(model,batch_size=batch_size))  # compute validation accuracy


              print(("Epoch {}: Train acc: {} |"+"Validation acc: {}").format(epoch + 1,train_acc[-1],val_acc[-1]))
              checkpoint_num += 1
              model_path = "bs{0}_lr{1}_epoch{2}_checkpoint_{3}".format(batch_size,learning_rate,epoch,checkpoint_num)
              torch.save({
                  'checkpoint_num': checkpoint_num,
                  'epoch': epoch,
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  #'train_loss': train_losses,
                  #'val_loss':val_losses,
                  'saved_train_loss': saved_train_losses,
                  'saved_val_loss':saved_val_losses,
                  'saved_x':saved_x,
                  #'iters':iters,
                  'train_acc':train_acc,
                  'val_acc':val_acc
                  }, './drive/My Drive/resnet_model_saved/'+model_path+'.pth')


    # plotting
    plt.title("Training Curve")
    plt.plot(saved_x, saved_train_losses, label="Train")
    plt.plot(saved_x, saved_val_losses, label='Validation')
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()

    plt.title("Training Curve")
    plt.plot(saved_x, train_acc, label="Train")
    plt.plot(saved_x, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()
    
    end_time = time.time()
    duration = end_time - start_time

    print("Final Training Accuracy: {}".format(saved_train_acc[-1]))
    print("Final Validation Accuracy: {}".format(saved_val_acc[-1]))
    print ("Trained Duration: {} seconds".format(duration))

In [0]:
#res50_v1 = torchvision.models.resnet.ResNet(torchvision.models.resnet.Bottleneck,[3, 4, 6, 3],num_classes=32).cuda()
#train_net(res50_v1,batch_size=20, num_epochs=100, learning_rate = 0.006,weight_decay=0.0002)

In [0]:
res50_v1 = torchvision.models.resnet.ResNet(torchvision.models.resnet.Bottleneck,[3, 4, 6, 3],num_classes=32).cuda()
train_net_continue(res50_v1,batch_size=20, num_epochs=100, learning_rate = 0.006,weight_decay=0.0002)

processing iterration 0...
processing iterration 1...
processing iterration 2...
processing iterration 3...
processing iterration 4...
processing iterration 5...
processing iterration 6...
processing iterration 7...
processing iterration 8...
processing iterration 9...
processing iterration 10...
processing iterration 11...
processing iterration 12...
processing iterration 13...
processing iterration 14...
processing iterration 15...
processing iterration 16...
processing iterration 17...
processing iterration 18...
processing iterration 19...
processing iterration 20...
processing iterration 21...
processing iterration 22...
processing iterration 23...
processing iterration 24...
processing iterration 25...
processing iterration 26...
processing iterration 27...
processing iterration 28...
processing iterration 29...
processing iterration 30...
processing iterration 31...
processing iterration 32...
processing iterration 33...
processing iterration 34...
processing iterration 35...
pr